In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import os
import cv2
from tqdm import tqdm
from utils import get_2D_projections

In [3]:
# vol_img = sitk.ReadImage(r"C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\sample3Dimg.nrrd")
# ct_img = sitk.ReadImage(r"D:\U-CAN-Lymfom_A\Ashish_SampleResampling\pat2_CT_resample_3Dimg.nii.gz")

# img_name= r'C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\Sample 2D projections\pat2_CT_resample_3Dimg'
# get_2D_projections(ct_img,'CT','LT','mean',15.0,img_n=img_name)
# file_reader = sitk.ImageFileReader()
# file_reader.SetImageIO('NrrdImageIO')
# file_reader.SetFileName(r"C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\sample3Dimg.nrrd")
# file_reader.Execute()

# v_img = sitk.ReadImage(r"C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\CTbrain50.nrrd")
# save_path=r"C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\Sample 2D projections\CTbrain50"

v_img = sitk.ReadImage(r"F:\ucan_lymfom\Selected_for_UCAN_project\Resampled_Selected_SampleImages\distorted_sample3Dimg_.nii.gz")
if not os.path.exists(r"F:\ucan_lymfom\Selected_for_UCAN_project\Resampled_Selected_SampleImages\2d projections"):
    os.mkdir(r"F:\ucan_lymfom\Selected_for_UCAN_project\Resampled_Selected_SampleImages\2d projections")

save_path=r"F:\ucan_lymfom\Selected_for_UCAN_project\Resampled_Selected_SampleImages\2d projections\sample1"

get_2D_projections(v_img,'CT','A','mean',15,img_n=save_path)

In [ ]:
# Display the image slice from the middle of the stack, z axis
# print(vol_img.GetDepth())
z = int(vol_img.GetDepth()/10)
npa_zslice = sitk.GetArrayViewFromImage(vol_img)[z,:,:]
npa = sitk.GetArrayViewFromImage(vol_img)
# Three plots displaying the same data, how do we deal with the high dynamic range?
fig = plt.figure(figsize=(10,3))

fig.add_subplot(1,3,1)
plt.imshow(npa_zslice)
plt.title('default colormap', fontsize=10)
plt.axis('off')

fig.add_subplot(1,3,2)
plt.imshow(npa_zslice,cmap=plt.cm.Greys_r)
plt.title('grey colormap', fontsize=10)
plt.axis('off')

fig.add_subplot(1,3,3)
plt.title('grey colormap,\n scaling based on volumetric min and max values', fontsize=10)
plt.imshow(npa_zslice,cmap=plt.cm.Greys_r, vmin=npa.min(), vmax=npa.max())
plt.axis('off')

In [ ]:
#stik [x,y,z]-> numpy [z,x,y]
nda = sitk.GetArrayFromImage(vol_img)
print(vol_img.GetSize())
print(nda.shape)

nda2d=sitk.GetArrayFromImage(vol_img[:,:,0])
print(nda2d)

In [ ]:
# numpy array  -> stik

nda_as_image = sitk.GetImageFromArray(nda)
print(nda_as_image.GetSize())

In [ ]:
# voxel queries

print(vol_img.GetPixelIDValue())
print(vol_img.GetPixelIDTypeAsString())
print(vol_img.GetNumberOfComponentsPerPixel())

In [ ]:
# Accessing and slicing
#help(vol_img.GetPixel)

#slice voxel first

# row z wise slice
# slice_2d=vol_img[:,:,50]
# plt.imshow(sitk.GetArrayFromImage(slice_2d),cmap='gray')

# row x wise slice
# slice_2d=vol_img[96,:,:]
# print(type(slice_2d))
# plt.imshow(np.add(sitk.GetArrayFromImage(slice_2d)),cmap='gray')

# row y wise slice
slice_2d=vol_img[:,40,:]
plt.imshow(sitk.GetArrayFromImage(slice_2d),cmap='gray')


In [ ]:
#display img averaged over all pixels

mean_img=np.zeros(nda_as_image.GetSize()[0:1])
for i in range (nda_as_image.GetSize()[0]):
   mean_img=mean_img+sitk.GetArrayFromImage(vol_img[i,:,:])
mean_img=mean_img/nda_as_image.GetSize()[0]
#mean_img=mean_img+50*np.ones(nda_as_image.GetSize()[0:1])
plt.imshow(mean_img,cmap='gray')

In [ ]:
#slice angle
plt.imshow(npa_zslice,cmap=plt.cm.Greys_r)

#type of projecttion - min/max

In [ ]:
def make_isotropic(image, interpolator = sitk.sitkLinear):
    '''
    Resample an image to isotropic pixels (using smallest spacing from original) and save to file. Many file formats 
    (jpg, png,...) expect the pixels to be isotropic. By default the function uses a linear interpolator. For
    label images one should use the sitkNearestNeighbor interpolator so as not to introduce non-existant labels.
    '''
    original_spacing = image.GetSpacing()
    # Image is already isotropic, just return a copy.
    if all(spc == original_spacing[0] for spc in original_spacing):
        return sitk.Image(image)
    # Make image isotropic via resampling.
    original_size = image.GetSize()
    min_spacing = min(original_spacing)
    new_spacing = [min_spacing]*image.GetDimension()
    new_size = [int(round(osz*ospc/min_spacing)) for osz,ospc in zip(original_size, original_spacing)]
    return sitk.Resample(image, new_size, sitk.Transform(), interpolator,
                         image.GetOrigin(), new_spacing, image.GetDirection(), 0,
                         image.GetPixelID())

def save_projections(image,img_name,type,max_intensity=50,min_intensity=-1024):
    writer = sitk.ImageFileWriter()
    img=sitk.Extract(image, image.GetSize())
    writer.SetFileName(img_name)

    #img=make_isotropic(img)

    if type=='PET':
        img_write=sitk.Cast(
            sitk.IntensityWindowing(
                img, windowMinimum=min_intensity, windowMaximum=max_intensity, outputMinimum=100.0, outputMaximum=255.0
            ),
            sitk.sitkUInt32,
        )
    else:
        img_write=sitk.Cast(
            img,
            sitk.sitkUInt32,
        )            

    writer.Execute(img_write)  #sitk.Cast(sitk.RescaleIntensity(img,outputMinimum=0,outputMaximum=15)

In [ ]:
projection = {'sum': sitk.SumProjection,
              'mean':  sitk.MeanProjection,
              'std': sitk.StandardDeviationProjection,
              'min': sitk.MinimumProjection,
              'max': sitk.MaximumProjection}
ptype = 'max'
paxis = 0

rotation_axis = [0,0,1]
rotation_angles = np.linspace(-1/2*np.pi, 1/2*np.pi, int(180.0/15.0)) #15 degree 
rotation_center = vol_img.TransformContinuousIndexToPhysicalPoint([(index-1)/2.0 for index in vol_img.GetSize()])

rotation_transform = sitk.VersorRigid3DTransform()
rotation_transform.SetCenter(rotation_center)

#Compute bounding box of rotating volume and the resampling grid structure
image_indexes = list(zip([0,0,0], [sz-1 for sz in vol_img.GetSize()]))
image_bounds = []
for i in image_indexes[0]:
    for j in image_indexes[1]:
        for k in image_indexes[2]:
            image_bounds.append(vol_img.TransformIndexToPhysicalPoint([i,j,k]))

all_points = []
for angle in rotation_angles:
    rotation_transform.SetRotation(rotation_axis, angle)    
    all_points.extend([rotation_transform.TransformPoint(pnt) for pnt in image_bounds])
    
all_points = np.array(all_points)
min_bounds = all_points.min(0)
max_bounds = all_points.max(0)

#resampling grid will be isotropic so no matter which direction we project to
#the images we save will always be isotropic (required for vol_img formats that 
#assume isotropy - jpg,png,tiff...)

new_spc = [np.min(vol_img.GetSpacing())]*3
new_sz = [int(sz/spc + 0.5) for spc,sz in zip(new_spc, max_bounds-min_bounds)]

pix_array=sitk.GetArrayFromImage(vol_img)
maxtensity,mintensity=pix_array.max(),pix_array.min()

proj_images = []
i=0
for angle in rotation_angles:
    rotation_transform.SetRotation(rotation_axis, angle) 
    resampled_image = sitk.Resample(image1=vol_img,
                                    size=new_sz,
                                    transform=rotation_transform,
                                    interpolator=sitk.sitkNearestNeighbor,
                                    outputOrigin=min_bounds,
                                    outputSpacing=new_spc,
                                    outputDirection = [1,0,0,0,1,0,0,0,1],
                                    defaultPixelValue =  0, #HU unit for air in CT, possibly set to 0 in other cases
                                    outputPixelType = vol_img.GetPixelID())
    proj_image = projection[ptype](resampled_image, paxis)
    extract_size = list(proj_image.GetSize())
    extract_size[paxis]=0
    proj_images.append(sitk.Extract(proj_image, extract_size))

    img_n=r'C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\Sample 2D projections\sample_{0}.png'.format(i)
    save_projections(sitk.InvertIntensity(sitk.Extract(proj_image, extract_size),maximum=1),img_name=img_n)
    print(i,angle,mintensity)
    i+=1


# Stack all images into fuax-volume for display
#sitk.Show(sitk.JoinSeries(proj_images))



In [ ]:
print(pix_array.min())

In [ ]:
#proj_image[0,:,:].GetSize()

inv_imgs=[]
for i in [0,3,9,11]:
    ext=sitk.Extract(proj_images[i], proj_images[i].GetSize())
    inv_ext=sitk.InvertIntensity(ext,maximum=1)
    
    inv_imgs.append(inv_ext)
    
#plt.imshow(sitk.GetArrayFromImage(proj_images[0]),cmap='gray')

intensity_clipper=sitk.IntensityWindowingImageFilter

for i in range(1,5):
    plt.subplot(2,4,i)
    intensity_clipper().SetOutputMaximum(15)
    intensity_clipper().SetOutputMinimum(0)
    plt.imshow(sitk.GetArrayFromImage(proj_images[i]),cmap='gray')

for i in range(1,5):
    plt.subplot(2,4,i+4)
    plt.imshow(sitk.GetArrayFromImage(inv_imgs[i-1]),cmap='gray')

In [ ]:
def generate_MIPs(SUV, rot_min=-90, rot_max=90, rot_interval=15 ):
    """
    Generate rotating 2D MIPs along coronal direction from (-90, 90).
    """
    imgs=[]
    SUV = np.transpose(SUV, (2,1,0))
    for i in tqdm(range(rot_min, rot_max+1, rot_interval)):
            print("angle: ", i)
            suv_temp = scipy.ndimage.rotate(SUV, angle=i, axes=(0,1))
            suv_temp = np.clip(suv_temp, 0, 15)
            suv_MIP = np.max(suv_temp, axis=1).astype("float")
            
            suv_MIP = suv_MIP/np.max(suv_MIP)
            suv_MIP = np.absolute(suv_MIP - np.amax(suv_MIP))
            suv_MIP = cv2.rotate(suv_MIP, cv2.ROTATE_90_COUNTERCLOCKWISE)

            suv_MIP = suv_MIP[:,60:-60]
            imgs.append(suv_MIP)
    return imgs
suv_imgs=generate_MIPs(sitk.GetArrayFromImage(vol_img))

In [ ]:
inv_suv_imgs=[]
for i in [0,4,10,12]:
    ext=sitk.Extract(sitk.GetImageFromArray(suv_imgs[i]), sitk.GetImageFromArray(suv_imgs[i]).GetSize())
    inv_ext=sitk.InvertIntensity(ext,maximum=1)
    inv_suv_imgs.append(inv_ext)
    #plt.imshow(sitk.GetImageFromArray(proj_image[0,:,:]),cmap='gray')
print(len(suv_imgs))
for i,j in zip([0,4,10,12],[1,2,3,4]):
    plt.subplot(1,4,j)
    plt.imshow(sitk.GetArrayFromImage(inv_suv_imgs[j-1]),cmap='gray')
    #plt.imshow(suv_imgs[i],cmap='gray')

In [ ]:
ct_vol= sitk.ReadImage(r"C:\Users\Audit\Uppsala - Masters Europe\Semester 3\Project in Image analysis - Software\UCAN-PET-CT-image-data-handling-pipeline\Task 2.1\Data\CTbrain50.nrrd")
skele=ct_vol <200 and ct_vol >50

#Thresholding
ct_vol.GetSize()
#adipose=vol_img[100<vol_img & 500>vol_img]

In [ ]:
skele.GetSize()

In [ ]:
image_viewer = sitk.ImageViewer()
sk=sitk.Extract(sitk.GetImageFromArray(skele), sitk.GetImageFromArray(vol_img).GetSize())
image_viewer.SetTitle('grid using ImageViewer class')
image_viewer.Execute(sk)

### Projections from selected images

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import os
import cv2
from tqdm import tqdm
import utils

In [ ]:
#read master data
master_data = pd.read_csv('F:/U-CAN-Lymfom_A/Selected_for_UCAN_project/Selected_exams_from_U-CAN-Lymfom_A_MasterDataset_2Oct2023.csv')

master_data.head(1)

In [ ]:
def make_mips(image, output_dir=''):
    #image = sitk.ReadImage(image_path)
    image_size = image.GetSize()

    #basename = os.path.basename(image_path)
    #if not os.path.isdir(output_dir):
    #    os.makedirs(output_dir)

    for dim in range(3):
        projection = sitk.MeanProjection(image, dim)

        if image_size[dim] % 2:  # odd number
            voxel = [0, 0, 0]
            voxel[dim] = (image_size[dim] - 1) / 2
            origin = image.TransformIndexToPhysicalPoint(voxel)
        else:  # even
            voxel1 = np.array([0, 0, 0], int)
            voxel2 = np.array([0, 0, 0], int)
            voxel1[dim] = image_size[dim] / 2 - 1
            voxel2[dim] = image_size[dim] / 2
            point1 = np.array(image.TransformIndexToPhysicalPoint(voxel1.tolist()))
            point2 = np.array(image.TransformIndexToPhysicalPoint(voxel2.tolist()))
            origin = np.mean(np.vstack((point1, point2)), 0)
        projection.SetOrigin(origin)
        projection.SetDirection(image.GetDirection())
        #proj_basename = basename.replace('.nii.gz', '_mip_{}.nii.gz'.format(dim))
        #plt.imshow(sitk.GetArrayFromImage(sitk.InvertIntensity(projection)), cmap='gray')
        sitk.WriteImage(projection, r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Selected_CTMean&PETMax_Projections\sample.nii.gz')#os.path.join(output_dir, proj_basename))

In [ ]:
img = sitk.ReadImage(r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Selected_CTMean&PETMax_Projections\sample.nii.gz')
plt.imshow(sitk.GetArrayFromImage(img))

In [ ]:
exception_lst = []
new_size = [384, 384, 384]
new_spacing = [2, 2, 3]
for index, row in master_data.iterrows():

    #vol_img = sitk.ReadImage(row['directory'])
    
    print(row['directory'])

    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(row['directory'])
    reader.SetFileNames(dicom_names)
    vol_img = reader.Execute()

    vol_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                         vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                         vol_img.GetPixelID())
    
    print(vol_img.GetSize())
    name_suffix = r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Selected_CTMean&PETMax_Projections\\' + \
                  str(row['patient_directory']) + str(row['modality'])
    if row['modality']=="PT":
        print("PT: ", row['modality'])
        ptype = 'max'
        maxpro = sitk.MaximumProjection(vol_img, 0)
        #plt.imshow(sitk.GetArrayFromImage(maxpro), cmap='gray')
    else:
        print("CT: ", row['modality'])
        ptype = 'mean'
        meanpro = sitk.MeanProjection(vol_img, 0)
        make_mips(vol_img)
        #plt.imshow(sitk.GetArrayFromImage(sitk.InvertIntensity(meanpro)), cmap='gray')#, vmin=50, vmax=255)
    
    angle = 90.0
    
    # try:
    #     utils.get_2D_projections(vol_img,ptype,angle,save_img=True,img_n=name_suffix, modality=row['modality'])
    # except:
    #     exception_lst.append(row['directory'])
    
    if index>0:
        break


In [ ]:
with open(r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Selected_CTMean&PETMax_Projections\exception_lst.txt', 'w') as fp:
    for item in exception_lst:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
print(master_data['modality'][0])
#master_data['modality'][0].strip()=="'CT'"
